In [2]:
import pandas as pd
import numpy as np
import os
from utils2.Date_Processor import *
from utils2.Data_run import arg_set, model_run
from datetime import datetime, timedelta


### 데이터를 1년, 2년, 3년 단위 및 예측기간(3월13~3월19)을 6시간 단위로 쪼갠 기간의 직전 3년 데이터로 나눔

In [6]:
temp_list_1nd = range(2012,2023,1)
temp_list_2nd = [2012,2014,2016,2018,2020,2021]  # 마지막이 2022~2023되는 문제가 있어서 마지막만 2021로 바꿈
temp_list_3nd = [2012,2015,2018,2020,2021]  # 마지막이 2021~2023되는 문제가 있어서, 2020을 추가함.

In [ ]:
# 1년 단위 데이터 생성

for i in temp_list_3nd:
    year_interval = 3
    start_year = i
    end_year = start_year+year_interval
    
    date_filter = DateFilter(result_df, date_column='date')
    filtered_df = date_filter.filter_by_date(f'{start_year}-06-01', f'{end_year}-06-01')
    filtered_df.reset_index(inplace=True, drop=True)
    filtered_df = filtered_df.iloc[:-1]
    filtered_df.to_csv(f'./Data_Final(실제 모델 입력데이터)/3_year/{start_year}_{end_year}_{year_interval}_result.csv', index=False)

In [ ]:
result_df = pd.read_csv('./Data_Preprocessed(전처리 데이터)/Final_Preprocessed_Data.csv')
start_time = '2020-03-13 00:00:00'
end_time = '2023-03-12 23:55:00'

move_time = timedelta(hours=6)
date_format = '%Y-%m-%d %H:%M:%S'
start_datetime = datetime.strptime(start_time, date_format)
end_datetime = datetime.strptime(end_time, date_format)

for i in range(28):
    start = str(start_datetime)
    end = str(end_datetime)
    start_name = start.replace(' ', '_').replace(':', '-')
    end_name = end.replace(' ', '_').replace(':', '-')
    print(start, end)
    
    date_filter = DateFilter(result_df, date_column='date')
    filtered_df = date_filter.filter_by_date(start, end)
    filtered_df.reset_index(inplace=True, drop=True)
    filtered_df.to_csv("./Data_Final(실제 모델 입력데이터)/predict_period/{}__{}_result.csv" .format(start_name, end_name), index=False)
    
    start_datetime = start_datetime + move_time
    end_datetime = end_datetime + move_time

### 예측 데이터를 바탕으로 실제 예측치오차(MAPE를 산출함)

In [52]:
total_df = pd.read_csv('./Data_Preprocessed(전처리 데이터)/Final_Preprocessed_Data.csv')

In [71]:
data_folder_list = os.listdir('./results/')

colab_list = [element for element in data_folder_list if 'colab' in element]
local_list = [element for element in data_folder_list if 'colab' not in element]
predict_local_list = [element for element in data_folder_list if 'ploc' in element]
predict_colab_list = [element for element in data_folder_list if 'cloc' in element]

In [49]:
local_list_1 = [element for element in local_list if '_1_' in element]
local_list_2 = [element for element in local_list if '_2_' in element]
local_list_3 = [element for element in local_list if '_3_' in element]
colab_list_1 = [element for element in colab_list if '_1_' in element]
colab_list_2 = [element for element in colab_list if '_2_' in element]
colab_list_3 = [element for element in colab_list if '_3_' in element]

In [69]:
def MAPE_search(folder_list):
    for i in folder_list:
        name = i.split('.csv')[0]
        data_path = './results/' + i + '/real_prediction.csv'
        temp_df = pd.read_csv(data_path)
        mean_mape = error_cal(total_df, temp_df, method='mape')
        
        print(f'File_Name : {name}, Mean_MAPE : {mean_mape}')
    

In [73]:
MAPE_search(predict_colab_list)

File_Name : 2020-03-13_00-00-00__2023-03-12_23-55-00_result, Mean_MAPE : 0.013163233671973928
File_Name : 2020-03-13_06-00-00__2023-03-13_05-55-00_result, Mean_MAPE : 0.10109957012318693
File_Name : 2020-03-13_12-00-00__2023-03-13_11-55-00_result, Mean_MAPE : 0.04256190330692964
File_Name : 2020-03-13_18-00-00__2023-03-13_17-55-00_result, Mean_MAPE : 0.03424410178201191
File_Name : 2020-03-14_00-00-00__2023-03-13_23-55-00_result, Mean_MAPE : 0.017099101380496258
File_Name : 2020-03-14_06-00-00__2023-03-14_05-55-00_result, Mean_MAPE : 0.04050252808147262
File_Name : 2020-03-14_12-00-00__2023-03-14_11-55-00_result, Mean_MAPE : 0.024250503033891178
File_Name : 2020-03-14_18-00-00__2023-03-14_17-55-00_result, Mean_MAPE : 0.010685208464378447
File_Name : 2020-03-15_00-00-00__2023-03-14_23-55-00_result, Mean_MAPE : 0.02001916834738091
File_Name : 2020-03-15_06-00-00__2023-03-15_05-55-00_result, Mean_MAPE : 0.01875183741772896
File_Name : 2020-03-15_12-00-00__2023-03-15_11-55-00_result, Mean_

In [74]:
# 여기부터는 6시간 리스트를 가져와서 하나로 합친 뒤 기술하는 것임.

actual = total_df.copy()

data_frame = pd.DataFrame()
for i in predict_colab_list:
    data_path = './results/' + i + '/real_prediction.csv'
    temp_df = pd.read_csv(data_path)
    data_frame = pd.concat([data_frame, temp_df], ignore_index=True)

merged_df = pd.merge(actual, data_frame, on='date', how='outer')
merged_df = merged_df.dropna()
merged_df = merged_df[['date', '현재수요(MW)_x', '현재수요(MW)_y']]
merged_df.reset_index(inplace=True, drop=True)
merged_df['Error(실제-예측)'] = merged_df['현재수요(MW)_x'] - merged_df['현재수요(MW)_y']
merged_df.to_csv('./Data_Result(결과 데이터)/result_colab.csv', index=False)

In [75]:
merged_df

,date,현재수요(MW)_x,현재수요(MW)_y,Error(실제-예측)
0,2023-03-13 00:00:00,58782.0,58634.015625,147.984375
1,2023-03-13 00:05:00,58954.4,58735.445312,218.954688
2,2023-03-13 00:10:00,58710.6,58289.500000,421.100000
3,2023-03-13 00:15:00,58298.4,58028.527344,269.872656
4,2023-03-13 00:20:00,58263.7,58086.558594,177.141406
...,...,...,...,...
2011,2023-03-19 23:35:00,58627.2,59258.433594,-631.233594
2012,2023-03-19 23:40:00,57941.2,59166.769531,-1225.569531
2013,2023-03-19 23:45:00,57726.2,58653.957031,-927.757031
2014,2023-03-19 23:50:00,57755.4,58611.851562,-856.451562


In [76]:
((abs(merged_df['현재수요(MW)_x'] - merged_df['현재수요(MW)_y'])) / merged_df['현재수요(MW)_x']).mean()

0.03800124717195253